In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('diabetes.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [8]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [9]:
df.Outcome.unique()

array([1, 0])

In [10]:
df_copy = df.copy()

In [11]:
df_copy.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [12]:
X = df_copy.iloc[:,:-1]
y = df_copy.iloc[:,-1]

In [13]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [14]:
y.head()

,Outcome
0,1
1,0
2,1
3,0
4,1


In [18]:
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((614, 8), (154, 8), (614,), (154,))

In [20]:
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [68]:
from sklearn.linear_model import LogisticRegression
parameter = {'penalty' : ('l1','l2','elasticnet',None),'C' : [1,10,20],'multi_class' : ('multinomial','OneVsRestClassifier')}
rscv = RandomizedSearchCV(LogisticRegression(),parameter,cv = 5,n_iter = 10)
rscv.fit(scaled_X_train,y_train)
rscv.best_params_

{'penalty': None, 'multi_class': 'multinomial', 'C': 1}

In [69]:
classifier = LogisticRegression(multi_class = 'multinomial', penalty = None, C = 1,solver = 'lbfgs')
classifier.fit(scaled_X_train,y_train)
y_pred = classifier.predict(scaled_X_test)

In [70]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [71]:
print(confusion_matrix(y_test,y_pred))

[[79 20]
 [18 37]]


In [72]:
print(accuracy_score(y_test,y_pred))

0.7532467532467533


In [73]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.81      0.80      0.81        99
           1       0.65      0.67      0.66        55

    accuracy                           0.75       154
   macro avg       0.73      0.74      0.73       154
weighted avg       0.76      0.75      0.75       154



In [74]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
parameter = {'penalty' : ('l1','l2','elasticnet',None),'C' : [1,10,20],'multi_class' : ('multinomial','OneVsRestClassifier')}
gscv = GridSearchCV(LogisticRegression(),parameter,cv = 5)
rscv.fit(scaled_X_train,y_train)
rscv.best_params_

{'penalty': None, 'multi_class': 'multinomial', 'C': 10}

In [75]:
classifier1 = LogisticRegression(multi_class = 'multinomial', penalty = None, C = 10,solver = 'lbfgs')
classifier1.fit(scaled_X_train,y_train)
y_pred = classifier1.predict(scaled_X_test)

In [76]:
print(confusion_matrix(y_test,y_pred))

[[79 20]
 [18 37]]


In [77]:
print(accuracy_score(y_test,y_pred))

0.7532467532467533


In [78]:
import pickle
pickle.dump(classifier,open('model.pkl','wb'))
pickle.dump(scaler,open('scaler.pkl','wb'))